In [1]:
import geopandas as gpd
import folium
import pandas as pd
import branca
from folium.plugins import MarkerCluster

import numpy as np
import matplotlib.pyplot as plt

In [144]:
gdfObservations = gpd.read_file('observations/observationsArgentina.shp')
gdfPolygons =  gpd.read_file('polygons/polygonsArgentina.shp', index=False)

In [145]:
gdfObservations

,id,observed_o,image_url,scientific,polygon,geometry
0,85336,2012-05-26,https://static.inaturalist.org/photos/124930/m...,Coprinus comatus,146.0,POINT (-58.48160 -34.48611)
1,364863,2009-11-21,https://inaturalist-open-data.s3.amazonaws.com...,Distantes,525.0,POINT (-68.09893 -54.81839)
2,367483,2009-11-21,https://inaturalist-open-data.s3.amazonaws.com...,Usnea,525.0,POINT (-68.55503 -54.84104)
3,565162,2013-12-22,https://inaturalist-open-data.s3.amazonaws.com...,Cyttaria hariotii,525.0,POINT (-68.48139 -54.84627)
4,1085403,2014-05-24,https://inaturalist-open-data.s3.amazonaws.com...,Gymnopilus,125.0,POINT (-58.91321 -34.41850)
...,...,...,...,...,...,...
28834,175999092,2023-07-30,https://inaturalist-open-data.s3.amazonaws.com...,Fungi,486.0,POINT (-60.69719 -31.57264)
28835,176002284,2023-07-30,https://static.inaturalist.org/photos/30592904...,Agaricales,NaN,POINT (-58.47089 -34.51473)
28836,176013924,2023-07-26,https://inaturalist-open-data.s3.amazonaws.com...,Polyporales,376.0,POINT (-53.97560 -25.85740)
28837,176014117,2023-07-30,NaN,Fungi,319.0,POINT (-64.39187 -36.67812)


In [146]:
gdfPolygons

,link,codpcia,departamen,provincia,#obs,#Obs_1,geometry
0,02001,02,Comuna 1,Ciudad Autónoma de Buenos Aires,NaN,435.0,"POLYGON ((-58.37501 -34.57959, -58.36690 -34.5..."
1,02002,02,Comuna 2,Ciudad Autónoma de Buenos Aires,NaN,25.0,"POLYGON ((-58.40084 -34.57126, -58.40005 -34.5..."
2,02003,02,Comuna 3,Ciudad Autónoma de Buenos Aires,NaN,17.0,"POLYGON ((-58.39365 -34.60154, -58.39348 -34.6..."
3,02004,02,Comuna 4,Ciudad Autónoma de Buenos Aires,NaN,22.0,"POLYGON ((-58.35695 -34.61980, -58.35672 -34.6..."
4,06091,06,Berazategui,Buenos Aires,NaN,545.0,"POLYGON ((-58.01702 -34.78120, -58.08380 -34.8..."
...,...,...,...,...,...,...,...
522,90112,90,Trancas,Tucumán,NaN,6.0,"POLYGON ((-65.54407 -26.09089, -65.54315 -26.0..."
523,90119,90,Yerba Buena,Tucumán,NaN,383.0,"POLYGON ((-65.25443 -26.78508, -65.25646 -26.7..."
524,94007,94,Río Grande,Tierra del Fuego,NaN,167.0,"POLYGON ((-68.60577 -54.55123, -68.60480 -54.3..."
525,94014,94,Ushuaia,Tierra del Fuego,NaN,2185.0,"MULTIPOLYGON (((-67.37727 -54.91089, -67.37819..."


In [140]:
# Function to generate the HTML content for the popup
def get_popup(polygon_id, current_publication = 0):
    publications = gdfObservations[gdfObservations["polygon"] == comuna-1].reset_index(drop=True)
    num_publications = len(publications)
    
    
    content = f"""
        <div id="publication_content">
           <p>Se han observado {num_publications} hongos en la Comuna {polygon_id}.</p>
        </div>
    """

    return content


In [141]:
m = folium.Map(location=[-34.6, -58.4],
           zoom_start=14,
           tiles='https://wms.ign.gob.ar/geoserver/gwc/service/tms/1.0.0/mapabase_gris@EPSG%3A3857@png/{z}/{x}/{-y}.png',
           attr='Paulo De Almeida')




# Create the colomap styles
colormap = branca.colormap.LinearColormap(
    vmin=gdfPolygons["#Obs"].quantile(0.0),
    vmax=gdfPolygons["#Obs"].quantile(1),
    colors = ['#FFFFCC', '#DAA520', '#A0522D', '#8B4513']
)


for comuna in range(1,16):
    
    polygon = folium.GeoJson(
                    gdfPolygons[gdfPolygons["COMUNA"] == comuna],
                    name = "CABA"+str(comuna),
                    style_function=lambda x: {
                        "fillColor": colormap(x["properties"]["#Obs"])
                        if x["properties"]["#Obs"] is not None
                        else "transparent",
                        "color": '#FFFFCC',
                        "dashArray": '3',
                        "fillOpacity": 0.55,
                        }, 
                    highlight_function=lambda x: {'fillColor': '#8B0000'},                      
                    zoom_on_click = True,
                    popup = folium.Popup(html =  get_popup(comuna-1),
                                         max_width =250, 
                                         style = {"fillColor" : '#FFFFFC'})
                    )
    polygon.add_to(m)
    

    
colormap.caption = 'Cantidad de hongos vistos'
m.add_child(colormap)



folium.raster_layers.TileLayer('Stamen Watercolor').add_to(m)
folium.LayerControl().add_to(m)





In [142]:
m

In [143]:
m.save("map.html")